# AI 모델 활용 1주차

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

d:\Anaconda3_envs\SpartaCodingClub\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch

print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

2.5.1+cu124
12.4
True


device(type='cuda')

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [5]:
text = "My name is"
encoded_input = tokenizer(text, return_tensors='pt')

In [6]:
encoded_input

{'input_ids': tensor([[3666, 1438,  318]]), 'attention_mask': tensor([[1, 1, 1]])}

In [7]:
output = model.generate(encoded_input['input_ids'], max_length=50)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [8]:
generated_text

"My name is John. I'm a man of God. I'm a man of God. I'm a man of God. I'm a man of God. I'm a man of God. I'm a man of God. I'm a"

In [9]:
output

tensor([[3666, 1438,  318, 1757,   13,  314, 1101,  257,  582,  286, 1793,   13,
          314, 1101,  257,  582,  286, 1793,   13,  314, 1101,  257,  582,  286,
         1793,   13,  314, 1101,  257,  582,  286, 1793,   13,  314, 1101,  257,
          582,  286, 1793,   13,  314, 1101,  257,  582,  286, 1793,   13,  314,
         1101,  257]])

# AI 모델 활용 2주차

### Pytorch를 활용하여 Transformer 모델 구현하기

In [10]:
import torch
import torch.nn as nn
from torch.nn import Transformer

In [11]:
model = Transformer(d_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6)

d:\Anaconda3_envs\SpartaCodingClub\lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [12]:
# 모델 학습 준비
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

In [13]:
# 모델 학습
"""
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(src, tgt)
    loss = criterion(output, tgt_labels)
    loss.backward()
    optimizer.step()

"""

'\nfor epoch in range(num_epochs):\n    optimizer.zero_grad()\n    output = model(src, tgt)\n    loss = criterion(output, tgt_labels)\n    loss.backward()\n    optimizer.step()\n\n'

In [14]:
# 사전 학습된 모델 활용하기
from transformers import AutoModelForCausalLM, AutoTokenizer

cache_path = 'D:\Anaconda3_envs\SpartaCodingClub\SpartaCodingClub\main_camp'

torch.hub.set_dir(cache_path)
model = AutoModelForCausalLM.from_pretrained('gpt2', cache_dir=cache_path)
tokenizer = AutoTokenizer.from_pretrained('gpt2', cache_dir=cache_path)

In [15]:
input_text = 'Once upon a time'
input_ids = tokenizer.encode(input_text, return_tensors='pt')

In [16]:
output = model.generate(input_ids, max_length=50, num_return_sequences = 1)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [17]:
tokenizer.decode(output[0], skip_special_tokens=True)

'Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a'

# AI 모델 활용 4주차

### GPT-2 모델로 텍스트 생성하기

In [18]:
from transformers import pipeline

# GPT-2 기반 텍스트 생성 파이프라인 로드
generator = pipeline("text-generation", model='gpt2', device=device)

# 텍스트 생성
generated_text = generator("Once upon a time", max_length=50, num_return_sequences=1, truncation=True)

# 결과 출력
print(generated_text[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Once upon a time the same happened to the ancient Egyptian, the Egyptian man who had long ago vanquished the evil gods is made a mortal for each of a man's deeds. Now it is clear that the same god is given immortality to a man that


In [24]:
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "너는 환영 인사를 하는 인공지능이야, 농담을 넣어 재미있게해줘"},
    {"role": "user", "content": "안녕?"}  
  ]
)

print("Assistant: " + completion.choices[0].message.content)

# 안녕하세요! 만나서 반가워요. 저랑 얘기하다가 재미 없으면 이렇게 생각해보세요: 적어도 엉덩이에 꼬리 달린 원숭이와는 다르게, 저는 평범하게 무리하지 않거든요! 뭐든 물어보세요, 도와드릴게요! 😄 

# 강사의 한마디: ....

Assistant: 안녕! 만나서 반가워! 혹시 여기 올 때 "전자우체국"에서 비행기로 오셨나요? 아니면 빛의 속도로 워프하셨나요?


In [2]:
from diffusers import StableDiffusionPipeline
import torch

# Stable Diffusion 파이프라인 로드
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipe = pipe.to('cuda')  # GPU 사용

# 텍스트 설명을 기반으로 이미지 생성
prompt = "A futuristic cityscape with flying cars at sunset"
image = pipe(prompt).images[0]

# 생성된 이미지 저장 및 출력
image.save("generated_image.png")
image.show()


100%|██████████| 50/50 [00:09<00:00,  5.53it/s]
